In [1]:
# SAGE MATH JUPYTER NOTEBOOK CoCALC Virtual Notebook Platform
def coppersmith_howgrave_univariate(pol, modulus, beta, mm, tt, XX):

    dd = pol.degree()
    nn = dd * mm + tt

    polZ = pol.change_ring(ZZ)
    x = polZ.parent().gen()

    # compute polynomials
    gg = []
    for ii in range(mm):
        for jj in range(dd):
            gg.append((x * XX)**jj * modulus**(mm - ii) * polZ(x * XX)**ii)
    for ii in range(tt):
        gg.append((x * XX)**ii * polZ(x * XX)**mm)

    # construct lattice B
    BB = Matrix(ZZ, nn)

    for ii in range(nn):
        for jj in range(ii+1):
            BB[ii, jj] = gg[ii][jj]

    # LLL
    BB = BB.LLL()

    # transform shortest vector in polynomial
    new_pol = 0
    for ii in range(nn):
        new_pol += x**ii * BB[0, ii] / XX**ii

    # factor polynomial
    potential_roots = new_pol.roots()

    # test roots
    roots = []
    for root in potential_roots:
        if root[0].is_integer():
            result = polZ(ZZ(root[0]))
            if gcd(modulus, result) >= modulus^beta:
                return (ZZ(root[0]))

    return roots

# Reference : https://www.cryptologie.net/article/222/implementation-of-coppersmith-attack-rsa-attack-using-lattice-reductions/
# Reference : CopperSmith Sage  https://github.com/mimoo/RSA-and-LLL-attacks

In [2]:
e = 5
N = 84364443735725034864402554533826279174703893439763343343863260342756678609216895093779263028809246505955647572176682669445270008816481771701417554768871285020442403001649254405058303439906229201909599348669565697534331652019516409514800265887388539283381053937433496994442146419682027649079704982600857517093
C = 23701787746829110396789094907319830305538180376427283226295906585301889543996533410539381779684366880970896279018807100530176651625086988655210858554133345906272561027798171440923147960165094891980452757852685707020289384698322665347609905744582248157246932007978339129630067022987966706955482598869800151693
pad = "You see a Gold-Bug in one corner. It is the key to a treasure found by"
msg_len = 100

pad_str = ''
for x in pad:
    pad_str = pad_str + bin(ord(x))[2:].zfill(8)

In [3]:
ZmodN = Zmod(N);
P.<M> = PolynomialRing(ZmodN);

for length_M in range(0, msg_len+1, 4):

    pol = ((int(pad_str, 2)<<length_M) + M)^e - C
    dd = pol.degree()

    # Tweak those
    beta    = 1
    epsilon = beta / 7
    mm      = ceil( beta**2 / (dd * epsilon) )
    tt      = floor( dd * mm * ((1/beta) - 1) )
    XX      = ceil( N**((beta**2/dd) - epsilon) )

    root = coppersmith_howgrave_univariate(pol, N, beta, mm, tt, XX)

    if root != []:
        break

In [4]:
root_binary = bin(root)[2:]
print("root:", root)
print("root_binary:", root_binary)

root: 595069740817087294497
root_binary: 1000000100001001000000011010000111010101100010010000010110110000100001


In [5]:
root_binary = '00' + root_binary #padding extra 2 bits
root_binary

'001000000100001001000000011010000111010101100010010000010110110000100001'

In [8]:
for i in range(0, len(root_binary), 8):
    print( chr( int( root_binary[i:i+8], 2) ) , end = '')

 B@hubAl!